In [1]:
from xdl import XDL

xdl_obj = XDL('van_leusen_oxazole.xdl')

In [2]:
print(xdl_obj.human_readable())

1) Heat/Chill reactor to 25°C with stirring.
2) Add tosmic (4.5 mL) to reactor.
3) Add 4-formylbenzonitrile (4.1 mL) to reactor.
4) Add methanol (1.4 mL) to reactor.
5) Stir reactor for 1.3333 hrs at 250 RPM.
6) Stop heating/chilling reactor.



In [3]:
xdl_obj.prepare_for_execution('mini_AH.json')

XDL:2020-03-12 12:00:20,365: Hardware is compatible




Review suggested vessel cleaning steps? (y, [n])
n
Verify solvents used in backbone and vessel cleaning? (y, [n])
n


XDL:2020-03-12 12:00:23,582:     Experiment Details

XDL:2020-03-12 12:00:23,582:            Reagent Volumes        
XDL:2020-03-12 12:00:23,582:     ------------------------------
XDL:2020-03-12 12:00:23,582:           methanol       | 31.4 mL
XDL:2020-03-12 12:00:23,582:            argon         | 15 mL
XDL:2020-03-12 12:00:23,582:            tosmic        | 7.5 mL
XDL:2020-03-12 12:00:23,582:     4-formylbenzonitrile | 7.1 mL
XDL:2020-03-12 12:00:23,582: 

XDL:2020-03-12 12:00:23,582:     Estimated duration: 1 hrs 21 mins


In [4]:
for base_step in xdl_obj.steps[1].base_steps:
    print(base_step, '\n', base_step.properties, '\n')

 {'from_vessel': 'flask_methanol', 'to_vessel': 'waste_1', 'volume': 3, 'move_speed': 40, 'aspiration_speed': 10, 'dispense_speed': 40, 'from_port': 0, 'to_port': 0, 'unique': False, 'through': [], 'use_backbone': True} 

 {'from_vessel': 'flask_methanol', 'to_vessel': 'waste_2', 'volume': 3, 'move_speed': 40, 'aspiration_speed': 10, 'dispense_speed': 40, 'from_port': 0, 'to_port': 0, 'unique': False, 'through': [], 'use_backbone': True} 

 {'from_vessel': 'flask_methanol', 'to_vessel': 'waste_3', 'volume': 3, 'move_speed': 40, 'aspiration_speed': 10, 'dispense_speed': 40, 'from_port': 0, 'to_port': 0, 'unique': False, 'through': [], 'use_backbone': True} 



In [1]:
from chempiler import Chempiler
import ChemputerAPI

c = Chempiler(
    experiment_code="oxazole",
    output_dir="\logs",
    graph_file="mini_AH.json",
    device_modules=[ChemputerAPI],
    simulation=False,
)

Starting UDP keepalive broadcast...


In [6]:
xdl_obj.execute(c)

XDL:2020-03-12 12:01:18,458: 
Operation Tree
--------------

XDL:2020-03-12 12:01:18,458: CSetStirRate
XDL:2020-03-12 12:01:18,458: CleanBackbone
XDL:2020-03-12 12:01:18,458:   CMove
XDL:2020-03-12 12:01:18,458:   CMove
XDL:2020-03-12 12:01:18,458:   CMove
XDL:2020-03-12 12:01:18,458: HeatChillToTemp
XDL:2020-03-12 12:01:18,474:   StartStir
XDL:2020-03-12 12:01:18,474:     CStir
XDL:2020-03-12 12:01:18,474:     CSetStirRate
XDL:2020-03-12 12:01:18,474:   StartHeatChill
XDL:2020-03-12 12:01:18,474:     CStirrerSetTemp
XDL:2020-03-12 12:01:18,474:     CStirrerHeat
XDL:2020-03-12 12:01:18,474:   CSetRecordingSpeed
XDL:2020-03-12 12:01:18,474:   CStirrerWaitForTemp
XDL:2020-03-12 12:01:18,474:   CSetRecordingSpeed
XDL:2020-03-12 12:01:18,474: Add
XDL:2020-03-12 12:01:18,474:   StartStir
XDL:2020-03-12 12:01:18,474:     CStir
XDL:2020-03-12 12:01:18,474:     CSetStirRate
XDL:2020-03-12 12:01:18,474:   PrimePumpForAdd
XDL:2020-03-12 12:01:18,474:     CMove
XDL:2020-03-12 12:01:18,474:   CMov

2020-03-12 12:05:28,616 ; pump_execution ; Moving 3 mL from flask_methanol (0) to waste_1 (0). (Speed - Initial: 10, Mid: 40, End: 40)
XDL:2020-03-12 12:05:54,621: Executing:
  CMove
    from_vessel: flask_methanol
    to_vessel: waste_2
    volume: 3
    move_speed: 40
    aspiration_speed: 10
    dispense_speed: 40
    from_port: 0
    to_port: 0
    unique: False
    through: []
    use_backbone: True

2020-03-12 12:05:54,621 ; pump_execution ; Moving 3 mL from flask_methanol (0) to waste_2 (0). (Speed - Initial: 10, Mid: 40, End: 40)
XDL:2020-03-12 12:06:28,514: Executing:
  CMove
    from_vessel: flask_methanol
    to_vessel: waste_3
    volume: 3
    move_speed: 40
    aspiration_speed: 10
    dispense_speed: 40
    from_port: 0
    to_port: 0
    unique: False
    through: []
    use_backbone: True

2020-03-12 12:06:28,514 ; pump_execution ; Moving 3 mL from flask_methanol (0) to waste_3 (0). (Speed - Initial: 10, Mid: 40, End: 40)
XDL:2020-03-12 12:07:09,195: Add
XDL:2020-03-12

response = self.tcp.recv(BUFFER_SIZE).decode()
OSError: [WinError 10051] A socket operation was attempted to an unreachable network
    response = self.tcp.recv(BUFFER_SIZE).decode()
OSError: [WinError 10051] A socket operation was attempted to an unreachable network
    response = self.tcp.recv(BUFFER_SIZE).decode()
OSError: [WinError 10051] A socket operation was attempted to an unreachable network
    response = self.tcp.recv(BUFFER_SIZE).decode()
OSError: [WinError 10051] A socket operation was attempted to an unreachable network
response = self.tcp.recv(BUFFER_SIZE).decode()
OSError: [WinError 10051] A socket operation was attempted to an unreachable network
2020-03-12 13:57:08,991 ; chempiler ; Waiting done.
XDL:2020-03-12 13:57:08,991: Executing:
  CSetRecordingSpeed
    recording_speed: 14

XDL:2020-03-12 13:57:08,991: Executing:
  StopStir
    vessel: reactor
    vessel_type: reactor
    vessel_has_stirrer: False

XDL:2020-03-12 13:57:08,991: StopHeatChill
XDL:2020-03-12 13:57

Empty: Reply queue timeout!

In [2]:
c.move("argon", "waste_3", 5, speed=50)

2020-03-12 14:04:57,599 ; pump_execution ; Moving 5 mL from argon (0) to waste_3 (0). (Speed - Initial: 50, Mid: 50, End: 50)


[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e760>,
   {'cmd': ('route', 3, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5832df0>,
   {'cmd': ('sink', 0), 'volume': 5, 'speed': 50})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e760>,
   {'cmd': ('route', 1, -1)}),
  (<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 2, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5832df0>,
   {'cmd': ('source', 0), 'volume': 5, 'speed': 50}),
  (<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('sink', 0), 'volume': 5, 'speed': 50})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 1, -1)}),
  (<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585ee50>,
   {'cmd': ('route', 2, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('source', 0), 'volume': 5, 'speed': 50}),
  (<ChemputerAPI.pump_valve_api.Chempu

In [8]:
c.move("water", "waste_2", 5, speed=20)
c.move("water", "reactor", 5, speed=20)
c.move("argon", "reactor", 5, speed=20)

2020-03-12 14:08:35,282 ; pump_execution ; Moving 5 mL from water (0) to waste_2 (0). (Speed - Initial: 20, Mid: 20, End: 20)
2020-03-12 14:09:27,460 ; pump_execution ; Moving 5 mL from water (0) to reactor (0). (Speed - Initial: 20, Mid: 20, End: 20)
2020-03-12 14:10:20,108 ; pump_execution ; Moving 5 mL from argon (0) to reactor (0). (Speed - Initial: 20, Mid: 20, End: 20)


[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e760>,
   {'cmd': ('route', 3, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5832df0>,
   {'cmd': ('sink', 0), 'volume': 5, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e760>,
   {'cmd': ('route', 1, -1)}),
  (<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 2, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5832df0>,
   {'cmd': ('source', 0), 'volume': 5, 'speed': 20}),
  (<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('sink', 0), 'volume': 5, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 4, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('source', 0), 'volume': 5, 'speed': 20})]]

In [9]:
c.move("reactor", "waste_2", 1, speed=20)

2020-03-12 14:12:41,590 ; pump_execution ; Moving 1 mL from reactor (0) to waste_2 (0). (Speed - Initial: 20, Mid: 20, End: 20)


[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 4, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('sink', 0), 'volume': 1, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 0, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('source', 0), 'volume': 1, 'speed': 20})]]

In [11]:
c.move("reactor", "dilution", 1, speed=20)

2020-03-12 14:15:48,032 ; pump_execution ; Moving 1 mL from reactor (0) to dilution (0). (Speed - Initial: 20, Mid: 20, End: 20)


[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 4, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('sink', 0), 'volume': 1, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 5, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('source', 0), 'volume': 1, 'speed': 20})]]

In [12]:
c.move("argon", "dilution", 1, speed=20)

2020-03-12 14:16:42,445 ; pump_execution ; Moving 1 mL from argon (0) to dilution (0). (Speed - Initial: 20, Mid: 20, End: 20)


[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e760>,
   {'cmd': ('route', 3, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5832df0>,
   {'cmd': ('sink', 0), 'volume': 1, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e760>,
   {'cmd': ('route', 1, -1)}),
  (<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 2, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5832df0>,
   {'cmd': ('source', 0), 'volume': 1, 'speed': 20}),
  (<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('sink', 0), 'volume': 1, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585e7c0>,
   {'cmd': ('route', 5, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865580>,
   {'cmd': ('source', 0), 'volume': 1, 'speed': 20})]]

In [13]:
c.move("acetonitrile", "dilution", 9, speed=20)

2020-03-12 14:17:23,538 ; pump_execution ; Moving 9 mL from acetonitrile (0) to dilution (0). (Speed - Initial: 20, Mid: 20, End: 20)


[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585ee50>,
   {'cmd': ('route', 5, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865910>,
   {'cmd': ('sink', 0), 'volume': 5, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585ee50>,
   {'cmd': ('route', 3, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865910>,
   {'cmd': ('source', 0), 'volume': 5, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585ee50>,
   {'cmd': ('route', 5, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865910>,
   {'cmd': ('sink', 0), 'volume': 4, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585ee50>,
   {'cmd': ('route', 3, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865910>,
   {'cmd': ('source', 0), 'volume': 4, 'speed': 20})]]

In [14]:
c.move("dilution", "waste_3", 7, speed=20)
c.move("acetonitrile", "dilution", 9, speed=20)

2020-03-12 14:45:13,529 ; pump_execution ; Moving 7 mL from dilution (0) to waste_3 (0). (Speed - Initial: 20, Mid: 20, End: 20)
2020-03-12 14:46:01,719 ; pump_execution ; Moving 9 mL from acetonitrile (0) to dilution (0). (Speed - Initial: 20, Mid: 20, End: 20)


[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585ee50>,
   {'cmd': ('route', 5, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865910>,
   {'cmd': ('sink', 0), 'volume': 5, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585ee50>,
   {'cmd': ('route', 3, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865910>,
   {'cmd': ('source', 0), 'volume': 5, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585ee50>,
   {'cmd': ('route', 5, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865910>,
   {'cmd': ('sink', 0), 'volume': 4, 'speed': 20})],
 [(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x235d585ee50>,
   {'cmd': ('route', 3, -1)})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x235d5865910>,
   {'cmd': ('source', 0), 'volume': 4, 'speed': 20})]]

Exception in thread Exception in thread Exception in thread Exception in thread Exception in thread Exception in thread pump_3 TCP thread:
Traceback (most recent call last):
  File "C:\Users\group\AppData\Local\Continuum\miniconda3\envs\optimizer\lib\threading.py", line 932, in _bootstrap_inner
valve_3 TCP threadvalve_2 TCP threadpump_1 TCP thread:
Traceback (most recent call last):
  File "C:\Users\group\AppData\Local\Continuum\miniconda3\envs\optimizer\lib\threading.py", line 932, in _bootstrap_inner
valve_1 TCP threadpump_2 TCP thread:
Traceback (most recent call last):
  File "C:\Users\group\AppData\Local\Continuum\miniconda3\envs\optimizer\lib\threading.py", line 932, in _bootstrap_inner
:
Traceback (most recent call last):
  File "C:\Users\group\AppData\Local\Continuum\miniconda3\envs\optimizer\lib\threading.py", line 932, in _bootstrap_inner
:
Traceback (most recent call last):
  File "C:\Users\group\AppData\Local\Continuum\miniconda3\envs\optimizer\lib\threading.py", line 932, 